# Script to generate task frame images.

This notebook will generate a task frame images for Figure 3 and save them to
`./frames/`.

In [1]:
"""Imports."""

import numpy as np
from pathlib import Path
from pynwb import NWBHDF5IO
import ast
from PIL import Image

import sys
sys.path.append('../../../task_visualization')
import task_state as task_state_lib
import renderer as renderer_lib

In [2]:
"""Load data."""

SUBJECT = "Perle"
SESSION = "2022-06-01"
WRITE_DIR = Path("./frames")

# Load the data
nwb_file_path = Path(
    f"../../../cache_osf/data_for_figures/dandi_data/behavior/sub-{SUBJECT}/"
    f"sub-{SUBJECT}_ses-{SESSION}_behavior+task.nwb"
)

# Read data from nwb file
with NWBHDF5IO(nwb_file_path, "r") as io:
    print(f"Processing {nwb_file_path}")
    nwbfile = io.read()
    all_trials_df = nwbfile.trials.to_dataframe()
    all_display_df = nwbfile.intervals['display'].to_dataframe()


Processing ../../../cache_osf/data_for_figures/dandi_data/behavior/sub-Perle/sub-Perle_ses-2022-06-01_behavior+task.nwb


In [3]:
"""Get trial data."""

# Compute trial and display dataframes for a trial
trial_df = all_trials_df.loc[0]
display_df = all_display_df[
    (all_display_df['start_time'] > trial_df['start_time']) &
    (all_display_df['stop_time'] < trial_df['stop_time'])
].reset_index(drop=True)

# Get task state and renderer
stim_positions = ast.literal_eval(trial_df.stimulus_object_positions)
stim_identities = ast.literal_eval(trial_df.stimulus_object_identities)
stim_target_str = trial_df.stimulus_object_target
stim_target = stim_target_str[1:-1].split(", ")
stim_target = [x == 'true' for x in stim_target]
task_state = task_state_lib.get_task_state(
    stim_positions, stim_identities, stim_target)
background_indices = trial_df.background_indices
renderer = renderer_lib.Renderer()

# Extract sprites from task state and set their positions and identities
sprite_a = task_state['prey'][0]
sprite_b = task_state['prey'][1]
sprite_a.position = np.array([0.85, 0.5])
sprite_b.position = np.array([0.325, 0.19689])
sprite_a.metadata["id"] = "a"
sprite_b.metadata["id"] = "b"

In [4]:
"""Render frames."""

# Render the combined task state with both sprites
sprite_a.opacity = 255
sprite_b.opacity = 255
image = renderer(task_state, background_indices)
Image.fromarray(image).save(WRITE_DIR / "frame_combo.pdf")

# Render only sprite A
sprite_b.opacity = 0
image_a = renderer(task_state, background_indices)
Image.fromarray(image_a).save(WRITE_DIR / "frame_a.pdf")

# Render only sprite B
sprite_a.opacity = 0
sprite_b.opacity = 255
image_b = renderer(task_state, background_indices)
Image.fromarray(image_b).save(WRITE_DIR / "frame_b.pdf")